In [3]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")

from src import paths
from src.utils import prepare_pd_dataset_for_lineclass, tokenize_and_align_labels

import pandas as pd

from datasets import Dataset, DatasetDict, concatenate_datasets

import pandas as pd

This is a preprocessing step to label all the observations in seantis_kisim.csv, the dataset containing the reports in line splitted format.

In [4]:
# The dataset needs columns rid, text, class_agg. The text column contains one line of a report per row. 
# The class_agg column can have all Null values, it will be filled by the lineclass model.

df_all = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"midatams/seantis_kisim.csv")
df_all = df_all[["research_id", "text"]].rename(columns={"research_id": "rid"})
df_all["class_agg"] = None
df_all

,rid,text,class_agg
0,016B6D16-2BBA-4C05-A8E4-30F761C95813,Diagnosen allgemein,None
1,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT Schubförmige Multiple Sklerose EM 03.06...,None
2,016B6D16-2BBA-4C05-A8E4-30F761C95813,"INDENT 03/2021: klinisch nicht aktiv, radiolog...",None
3,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT 06/2019 Schub mit Hypästhesie der Beine...,None
4,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT MRI WS 06/2019: kontrastmittelaufnehmen...,None
...,...,...,...
5664,C3997B98-9A42-4A63-A6EE-FC2BEDB78D14,"INDENT Soor-Stomatitis, ED 07.02.2020",None
5665,C3997B98-9A42-4A63-A6EE-FC2BEDB78D14,"INDENT Bakterieller Harnwegsinfekt, ED 11.02.2020",None
5666,C3997B98-9A42-4A63-A6EE-FC2BEDB78D14,INDENT Stomatitis aphthosa mit schmerzhafter A...,None
5667,C3997B98-9A42-4A63-A6EE-FC2BEDB78D14,"INDENT Re-Soor-Stomatitis, ED 11.03.2020",None


In [5]:
df_train = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_clean_train.csv")
df_val = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_clean_val.csv")
df_test = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_clean_test.csv")

dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "val": Dataset.from_pandas(df_val),
    "test": Dataset.from_pandas(df_test),
    "all": Dataset.from_pandas(df_all)
})

df_token_train = prepare_pd_dataset_for_lineclass(df_train)
df_token_val = prepare_pd_dataset_for_lineclass(df_val)
df_token_test = prepare_pd_dataset_for_lineclass(df_test)
df_token_all = prepare_pd_dataset_for_lineclass(df_all)

dataset_token = DatasetDict({
    "train": Dataset.from_pandas(df_token_train),
    "val": Dataset.from_pandas(df_token_val),
    "test": Dataset.from_pandas(df_token_test),
    "all": Dataset.from_pandas(df_token_all)
})

# Add a few of the validation examples to the training set
dataset_token["train"] = concatenate_datasets([dataset_token["train"], dataset_token["val"].select(range(8))])
dataset_token["val"] = dataset_token["val"].select(range(8, len(dataset_token["val"])))

# Save the tokenized datasets
dataset_token.save_to_disk(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_for_token_classification")

Saving the dataset (0/1 shards):   0%|          | 0/55 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/299 [00:00<?, ? examples/s]